In [26]:
import os
from typing import List, Dict
from trulens.apps.custom import TruCustomApp
from trulens.core import TruSession
from trulens.core import Feedback
from trulens.providers.openai import OpenAI
from dotenv import load_dotenv
import google.generativeai as genai
from trulens.apps.custom import instrument



In [27]:
load_dotenv()

True

In [28]:
from trulens.core import TruSession


session = TruSession()
session.reset_database()

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [29]:
# pip install pinecone[grpc]
from pinecone.grpc import PineconeGRPC as Pinecone


pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("gemini-test")

In [30]:


genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

In [31]:
embed = genai.embed_content
embed_model_name = "models/text-embedding-004"
task_type="retrieval_document"

In [32]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924


In [33]:
prompt ="Given the provided context, generate a response that is accurate, concise, and strictly aligned with the information retrieved. Ensure the response does not include hallucinations, speculations, or unsupported claims. The response should be neutral, fact-based, and respectful, especially when addressing sensitive or ambiguous topics. If the context provided is insufficient, clearly state that more information is needed. Prioritize safety and relevance, and avoid generating offensive or harmful content."

In [53]:
llm = genai.GenerativeModel("models/gemini-1.5-pro-002",
                            system_instruction=prompt)


In [35]:
class retriever:
        def __init__(self, embed, embed_model_name, index):
             self.embed = embed
             self.embed_model_name = embed_model_name
             self.index = index
        def get_data(self,query):
            embedding=self.embed(
            model=self.embed_model_name,
            task_type=task_type,
            content= query)["embedding"]

            vecs = self.index.query(
            vector=embedding,
            top_k=5,
            includeMetadata=True,
            include_values=True
        )["matches"]
            ids=[] 
            for match in vecs:
                ids.append(match.id)
            data = self.index.fetch(ids)
            docs = []
            for key in data["vectors"]:
                docs.append(data["vectors"][key]["metadata"]["text"])
            return docs



In [54]:
class generate:
    def __init__(self, llm):
        self.llm = llm
    
    def generate(self, query, context):
        formatted_context = "\n".join([str(doc) for doc in context])
        response = self.llm.generate_content(query + formatted_context).text
        return response
        

In [55]:
ret = retriever(embed, embed_model_name, index)


In [56]:
class rag_gemini:
    def __init__(self,llm,retriever):
        self.retriever = retriever
        self.llm=llm
    @instrument
    def retrieve(self, query: str) -> List[str]:
        """
        Method to handle document retrieval.
        IMPORTANT: The method name 'retrieve' will be used in selectors
        """
        documents = self.retriever.get_data(query)
        return documents
    
    @instrument
    def generate(self, query: str, context: List[str]) -> str:
        """
        Method to handle response generation.
        IMPORTANT: The method name 'generate' will be used in selectors
        """
        formatted_context = "\n".join([str(doc) for doc in context])
        response = self.llm.generate_content(query + formatted_context).text
        return response
    
    @instrument
    def query(self, question: str) -> Dict:
        """
        Main method that orchestrates the RAG pipeline.
        IMPORTANT: Return keys must match selector paths
        """
        context = self.retrieve(question)
        response = self.generate(question, context)
        
        return response

In [39]:
rag_app = rag_gemini(llm, ret)

In [57]:
import numpy as np
from trulens.core import Feedback
from trulens.core import Select
from trulens.providers.openai import OpenAI

provider = OpenAI(model_engine="gpt-4o", api_key="OPEN_AI_EVAL_KEY")

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)
# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input()
    .on_output()
)

# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(Select.RecordCalls.retrieve.rets[:])
    .aggregate(np.mean)  # choose a different aggregation method if you wish
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets[:] .


In [58]:
import numpy as np
from trulens.core import Feedback
from trulens.core import Select
from trulens.providers.openai import OpenAI

provider = OpenAI(model_engine="gpt-4o", api_key=os.getenv("OPEN_AI_EVAL_KEY"))

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)
# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input()
    .on_output()
)

# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(Select.RecordCalls.retrieve.rets[:])
    .aggregate(np.mean)  # choose a different aggregation method if you wish
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets[:] .


In [62]:
from trulens.apps.custom import TruCustomApp

tru_rag = TruCustomApp(
    rag_app,
    app_name="RAG Gemini",
    app_version="Pro",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

In [63]:
HardQuestions=[{
    "question": "What specific condition must be met for a health facility to be eligible to apply for narcotic drug quotas from MOHAP's central warehouses?",
    "answer": "The facility must submit an official letter from the technical director, detailing the types, quantities, concentrations of narcotic drugs required, and the number of hospital beds and operations performed.",
    "context": "This service allows hospitals and one-day surgery centers to apply for narcotic drug quotas for use within the facility.",
    "source": "https://mohap.gov.ae/en/services/approve-narcotic-drugs-for-internal-pharmacies-at-private-hospitals"
  },
  {
    "question": "How many beds must a hospital have before it can apply for the higher licensing fees of AED 30,000 for MOHAP approval?",
    "answer": "Hospitals with 50 to 100 beds must pay AED 30,000 for final licensing fees.",
    "context": "This fee applies to hospitals with a specific number of beds when applying for final licensing approval from MOHAP.",
    "source": "https://mohap.gov.ae/en/services/licensing-of-private-medical-facilities"
  },
  {
    "question": "Which specific document must be completed at the Customer Happiness Centers by the owner of a healthcare facility applying for licensing?",
    "answer": "The owner must complete a character reference document, which is available at the Customer Happiness Centers.",
    "context": "This service requires a character reference for licensing a healthcare facility under MOHAP regulations.",
    "source": "https://mohap.gov.ae/en/services/licensing-of-private-medical-facilities"
  },
  {
    "question": "What is the exact document required from the Department of Economic Development for the final approval of a medical facility license?",
    "answer": "A trade license issued by the Department of Economic Development is required for the final approval of a medical facility license.",
    "context": "This document is part of the final approval process for healthcare facilities applying for a license under MOHAP.",
    "source": "https://mohap.gov.ae/en/services/licensing-of-private-medical-facilities"
  },
  {
    "question": "What is the specific service classification for the 'Appeal Against Medical Licensing Committee Decisions' service?",
    "answer": "The service classification for this appeal is 'Transactional' and the service type is 'Government to Business'.",
    "context": "This service allows businesses and professionals to appeal decisions made by the Medical Licensing Committee.",
    "source": "https://mohap.gov.ae/en/services/appeal-against-medical-licensing-committee-decisions"
  },
  {
    "question": "What is the minimum number of days it takes to receive engineering team approval after submitting a request for health facility inspection?",
    "answer": "It takes a minimum of 2 working days to receive engineering team approval after submitting an inspection request.",
    "context": "This service involves inspecting the facility before approval is granted for licensing.",
    "source": "https://mohap.gov.ae/en/services/licensing-of-private-medical-facilities"
  },
  {
    "question": "Which exact document must include the names of all licensed doctors and technicians within a healthcare facility when applying for a visiting doctor license?",
    "answer": "A list of licensed doctors and technicians in the facility must be included when applying for a visiting doctor license.",
    "context": "This is required when applying for the service allowing a visiting doctor to practice at a private healthcare facility.",
    "source": "https://mohap.gov.ae/en/services/licensing-of-a-visiting-doctor-from-the-uae"
  },
  {
    "question": "What is the exact fee for re-verifying a geometric site plan for a general clinic or medical center?",
    "answer": "The fee for re-verifying a geometric site plan for a general clinic or medical center is AED 500.",
    "context": "This service requires a fee for re-verifying geometric site plans during the healthcare facility licensing process.",
    "source": "https://mohap.gov.ae/en/services/licensing-of-private-medical-facilities"
  },
  {
    "question": "What specific exclusion applies to the issuance of a good professional conduct certificate for health professionals?",
    "answer": "A good professional conduct certificate will not be issued to those who have only an initial license from the Ministry of Health and Prevention.",
    "context": "This service issues good professional conduct certificates to licensed health professionals in the UAE.",
    "source": "https://mohap.gov.ae/en/services/issue-of-good-professional-conduct-certificates-for-health-professional"
  },
  {
    "question": "What exact documentation must be submitted if a health professional’s license has been canceled and they are applying for a good standing certificate?",
    "answer": "A license cancellation letter from MOHAP must be submitted when applying for a good standing certificate after license cancellation.",
    "context": "This service provides good standing certificates for health professionals who have held MOHAP licenses.",
    "source": "https://mohap.gov.ae/en/services/issue-of-good-professional-conduct-certificates-for-health-professional"
  },
  {
    "question": "What is the maximum number of specialties a diagnostic center can add before incurring only AED 100 per additional specialty?",
    "answer": "A diagnostic center can add up to 3 specialties before incurring AED 100 per additional specialty.",
    "context": "This service allows diagnostic centers to add new specialties, with specific fee structures depending on the number of specialties.",
    "source": "https://mohap.gov.ae/en/services/adding-a-new-specialty-to-private-health-facilities"
  },
  {
    "question": "For how long is the initial approval of a medical facility’s license valid, and what does it allow the owner to do?",
    "answer": "The initial approval is valid for 6 months and allows the owner to meet the technical requirements for obtaining a MOHAP license but does not permit the facility to practice or operate.",
    "context": "The initial approval grants time for the facility to complete technical requirements, but does not authorize operation.",
    "source": "https://mohap.gov.ae/en/services/licensing-of-private-medical-facilities"
  },
  {
    "question": "What specific document must be signed by the supervising pharmacist or doctor when applying for approval to modify narcotic drugs quotas?",
    "answer": "Form F6, 'Approval/Determination or Modification of Quotas of the Drugs,' must be signed by the supervising pharmacist or doctor.",
    "context": "This service allows hospitals and one-day surgery clinics to modify narcotic drug quotas for internal pharmacies.",
    "source": "https://mohap.gov.ae/en/services/approve-narcotic-drugs-for-internal-pharmacies-at-private-hospitals"
  },
  {
    "question": "How many transactions were processed for the service 'Approve Narcotic Drugs for Internal Pharmacies at Private Hospitals' according to MOHAP records?",
    "answer": "There were 8 transactions processed for this service according to MOHAP records.",
    "context": "This service allows hospitals and day surgery centers to apply for approval to purchase narcotic drugs.",
    "source": "https://mohap.gov.ae/en/services/approve-narcotic-drugs-for-internal-pharmacies-at-private-hospitals"
  },
  {
    "question": "What is the exact timeframe for submitting an appeal against a Medical Licensing Committee decision after receiving the decision?",
    "answer": "An appeal must be submitted within 15 days from the date of receiving the decision.",
    "context": "This service allows healthcare professionals and establishments to appeal against decisions made by the Medical Licensing Committee.",
    "source": "https://mohap.gov.ae/en/services/appeal-against-medical-licensing-committee-decisions"
  },
  {
    "question": "Which document must be presented at the General Directorate of Residency and Foreigners’ Affairs if parents choose to collect the newborn’s documents through the 'Mabrouk Ma Yak' service in person?",
    "answer": "Parents must present the original family data at the General Directorate of Residency and Foreigners’ Affairs when collecting the newborn’s documents.",
    "context": "'Mabrouk Ma Yak' provides newborn Emiratis with multiple identification documents, including options for collection and delivery.",
    "source": "https://mohap.gov.ae/en/about-us/projects-and-initiatives/mabrouk-ma-yak"
  },
  {
    "question": "What are the exact conditions under which a good professional conduct certificate cannot be issued to a health professional?",
    "answer": "A good professional conduct certificate cannot be issued to a trainee, a visitor, someone with an initial license, or anyone not licensed by the Ministry of Health and Prevention.",
    "context": "This service provides good professional conduct certificates for health professionals under specific conditions.",
    "source": "https://mohap.gov.ae/en/services/issue-of-good-professional-conduct-certificates-for-health-professional"
  },
  {
    "question": "What is the service classification and service type for 'Registration of a Medical Equipment' under MOHAP?",
    "answer": "The service classification is 'Transactional' and the service type is 'Government to Business.'",
    "context": "This service allows businesses to register medical equipment for use in healthcare facilities.",
    "source": "https://mohap.gov.ae/en/services/registration-of-a-medical-equipment"
  },
  {
    "question": "Which regulatory entity must issue a radiation protection license for healthcare facilities in the UAE?",
    "answer": "The Federal Authority for Nuclear Regulation must issue the radiation protection license.",
    "context": "Healthcare facilities requiring radiation services must obtain a license from the Federal Authority for Nuclear Regulation.",
    "source": "https://mohap.gov.ae/en/services/licensing-of-private-medical-facilities"
  },
  {
    "question": "What is the email address for submitting inquiries about controlled chemicals to MOHAP?",
    "answer": "The email address is controlledchemicals@mohap.gov.ae.",
    "context": "This contact information is provided for inquiries related to the regulation of anesthetic drugs and chemical precursors.",
    "source": "https://mohap.gov.ae/en/services/provide-controlled-drugs-prescription-book"
  }]

In [64]:
rag_app.query(
       HardQuestions[0]["question"]
    )

'The provided text describes various services offered by MOHAP related to narcotic drug quotas, but it does not explicitly state a specific condition for a health facility to be eligible to apply for narcotic drug quotas from MOHAP\'s central warehouses. \n\nThe text mentions that the "quantities of drugs expected to be used annually should be determined" as a condition or requirement for several of the services, but it does not specify whether this is the sole condition for eligibility to apply for quotas from MOHAP\'s central warehouses.  \n\nTo answer your question, more information is needed about the specific requirements for applying for narcotic drug quotas from MOHAP\'s central warehouses. \n'

In [65]:
with tru_rag as recording:
    for eval in HardQuestions[0:11]:
        rag_app.query(
        eval["question"]
    )



ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [66]:
from trulens.dashboard import run_dashboard

run_dashboard(session)

Starting dashboard ...
Dashboard already running at path:   Network URL: http://192.168.1.4:62576



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>